<a href="https://colab.research.google.com/github/Janith123gihan/Car-Damage-Classification/blob/main/Car_Damage_Classification_Transfer_Learning_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras import layers
import tensorflow as tf
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:

print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2.9.2
Found GPU at: /device:GPU:0


In [ ]:
def set_data(train,test, validation ,batchSize, image_size):
 
 Image_size = [image_size,image_size]

 train_datagen= ImageDataGenerator(rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,rescale=1./255,
                                   zoom_range=0.2,
                                   horizontal_flip=0.5, vertical_flip=0.5
                                   )

 test_datagen = ImageDataGenerator(rescale=1./255)

 val_datagen = ImageDataGenerator(rescale=1./255)

 #Training dataset
 train_set = train_datagen.flow_from_directory(
                train,
                target_size=Image_size,
                batch_size=batchSize, 
                color_mode="rgb",
                #shuffle=True,             
                interpolation='bicubic',
                class_mode='categorical'
                )
 #Testing dataset
 test_set= test_datagen.flow_from_directory(
              test,
              target_size=Image_size,
              color_mode = "rgb", interpolation='bicubic',
              class_mode='categorical', shuffle=False
             )
 validation_set = val_datagen.flow_from_directory(
    validation, # same directory as training data
    target_size=Image_size,
    color_mode = "rgb",interpolation='bicubic',
    batch_size=batchSize)
 return train_set, test_set, validation_set;

In [ ]:
def unfreeze_model(model, num_of_layers):
    for layer in model.layers[num_of_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
    return model

In [ ]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
from keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import BatchNormalization

base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape = (224,224,3)))

# head_model = Model(base_model.input, base_model.layers[-2].output)
# head_model.trainable = False
# head_model = unfreeze_model(head_model,-3)

# x = head_model.output
# se = layers.GlobalAveragePooling2D(name="ch_pool")(x)
# # se = layers.Reshape((1,1,512))(se)
# se = layers.Dense(32,activation="swish",kernel_initializer='he_normal', use_bias=False)(se)
# se = layers.Dense(512,activation="sigmoid",kernel_initializer='he_normal', use_bias=False)(se)
# # x = layers.Multiply()([se, x])
# x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.4)(x)
# x = layers.Dense(256, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.3)(x)

for layer in base_model.layers[:170]:
    layer.trainable = False

model = Sequential()
# model.add(K.layers.Lambda(lambda x: tf.image.resize(x, (224, 224))))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()












# outputs = layers.Dense(3, activation="softmax", name="pred")(x)
# model = tf.keras.Model(head_model.input, outputs)
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["categorical_accuracy"])
# model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                      

In [ ]:
def plot_hist(hist):
    plt.figure(3)
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
    plt.figure(4)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:

batchSize = 32
epoches =80;
image_size = 224;
train_path = '/content/drive/MyDrive/Car Damage Classification/training'
test_path = '/content/drive/MyDrive/Car Damage Classification/testing'
validation_path = '/content/drive/MyDrive/Car Damage Classification/validation'




train_set, test_set, validation_set = set_data(train_path,test_path,validation_path, batchSize, image_size)

history=model.fit(train_set,  validation_data= validation_set,epochs = epoches)
plot_hist(history)


Found 784 images belonging to 3 classes.
Found 195 images belonging to 3 classes.
Found 171 images belonging to 3 classes.
Epoch 1/80
25/25 [==============================] - 13s 521ms/step - loss: 0.8523 - accuracy: 0.6059 - val_loss: 2.1887 - val_accuracy: 0.4211
Epoch 2/80
25/25 [==============================] - 13s 509ms/step - loss: 0.8767 - accuracy: 0.5765 - val_loss: 1.6852 - val_accuracy: 0.4327
Epoch 3/80
25/25 [==============================] - 13s 509ms/step - loss: 0.8353 - accuracy: 0.6020 - val_loss: 1.0313 - val_accuracy: 0.5789
Epoch 4/80
25/25 [==============================] - 13s 508ms/step - loss: 0.8605 - accuracy: 0.6020 - val_loss: 1.0305 - val_accuracy: 0.5789
Epoch 5/80
25/25 [==============================] - 13s 512ms/step - loss: 0.8390 - accuracy: 0.6071 - val_loss: 1.4062 - val_accuracy: 0.4620
Epoch 6/80
25/25 [==============================] - 13s 503ms/step - loss: 0.8520 - accuracy: 0.5931 - val_loss: 1.1865 - val_accuracy: 0.5088
Epoch 7/80
25/25 [=

In [ ]:
model.save('CNN-Car_Damage_Classification.h5')

# Testing Model

In [ ]:

model.save('/content/drive/MyDrive/Car Damage Classification/my_model')
results = model.evaluate(test_set,batch_size=32)
accuracy = results[1]
predict_labels=model.predict(test_set,batch_size=batchSize)
test_labels=test_set.classes
print(accuracy)


7/7 [==============================] - 2s 136ms/step
0.4564102590084076


In [ ]:
print(test_labels)
print(predict_labels.argmax(axis=1))
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(test_labels, predict_labels.argmax(axis=1), target_names=['minor', 'moderate', 'severe']))
confusion = confusion_matrix(test_labels, predict_labels.argmax(axis=1))
print('Confusion Matrix\n')
print(confusion)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2]
[0 1 0 2 2 2 2 1 0 1 1 2 2 2 1 0 1 1 2 2 2 2 1 2 2 2 1 2 2 1 2 2 1 1 1 1 1
 1 2 1 0 1 1 2 1 2 1 1 1 1 2 2 1 1 2 2 2 1 1 1 2 2 1 1 2 2 2 1 2 2 2 1 2 1
 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 1 2 2 1 2 2 2 2 1 0 2 2 2 2 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2]

Classification Report

              precision    recall  f1-score   support

       minor       0.83      0.09      0.16        55
    moderate       0.31      0.24      0.27        63
      severe       